# TEAM-SASA     
### Initial POC and EDA for ABS-CBN Website

In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains

from time import sleep

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt 

## HELPER CLASS - WEBSCRAPING

In [7]:
DRIVER_PATH = 'chromedriver.exe'

class InquirerScraper:

    def __init__(self, search_url):
        self.urls = []
        self.search_url = search_url
        self.driver = webdriver.Chrome(DRIVER_PATH)

    def crawl(self):
        self.driver.get(self.search_url)
        totalpages = self.get_total_pages()
        self.get_pages(totalpages)

    def get_driver(self):
        return self.driver
        
    def get_urls(self):
        return self.urls

    def get_urls_count(self):
        return len(self.urls)

    def get_total_pages(self):
        pages = self.driver.find_elements(By.CSS_SELECTOR, 'div.gsc-cursor-page')
        return len(pages)

    def add_urls(self, page):
        print(f'Start crawling article URLs for page {page}...')
        alinks = self.driver.find_elements(By.CSS_SELECTOR, 'a.gs-title')
        for a in alinks:
            link = a.get_attribute('href')
            if (link != None) and (link not in self.urls):
                self.urls.append(link)
        print('Done.')

    def get_pages(self, totalpages):
        print(f'Total number of pages to crawl: {totalpages}', end='\n\n')
        # Page 1
        self.add_urls(1)
        # Page 2 onwards
        self.driver.maximize_window()
        for p in range(1, 11):
            '''
            self.driver.execute_script('window.scrollBy(0, document.body.scrollHeight)', '') 
            pages = self.driver.find_elements(By.CSS_SELECTOR, 'div.gsc-cursor-page')
            pages[p].click()
            '''
            url = "https://www.inquirer.net/search?q=covid%2019%20vaccines%20philippines#gsc.tab=0&gsc.q=covid%2019%20vaccines%20philippines&gsc.page=" + str(p+1)
            self.driver.get(url)
            sleep(5)
            self.add_urls(p+1)

    def close_popups(self):
        try:
            self.driver.find_element(By.XPATH, '//*[@id="survicate-box"]/div/div/div[3]/div[2]/button').click()
        except:
            pass
        try:
            self.driver.find_element(By.XPATH, '//*[@id="onesignal-slidedown-cancel-button"]').click()
        except:
            pass
        try:
            # Close footer ads
            self.driver.find_element(By.XPATH, '//*[@id="__next"]/div/div[16]//following::div/p/span').click()
        except:
            pass

    def scrape_data(self, url):
        ### PREPARE ARTICLE URL
        self.driver.get(url)
        sleep(15)

        ### PREPARE TITLE
        title = self.driver.find_element(By.CSS_SELECTOR, 'h1.entry-title')
        title = title.text
        sleep(10)
        print(f'\n[Title]\n{title}')

        ### PREPARE PUBLISHED DATE
        pubdate = self.driver.find_element(By.ID, 'art_plat')
        pubdate = pubdate.text
        sleep(10)
        print(f'\n[Date Pulished]\n{pubdate}')
        

        ### PREPARE AUTHOR NAME
        author = self.driver.find_element(By.XPATH, '//*[@id="art_author"]/span/a')
        author = author.text
        sleep(10)
        print(f'\n[Author]\n{author}')

        # Try closing popups before clicking 'SEE THE RESULTS' button
        sleep(2)
        self.close_popups()

        ### PREPARE ARTICLE CONTENT
        article = self.driver.find_element(By.ID, 'article_content')
        article_content = article.text.replace('ADVERTISEMENT','').replace('How did this story make you feel?', '').replace('INSPIRED','')\
            .replace('HAPPY','').replace('AMUSED','').replace('NEUTRAL','').replace('ANNOYED','').replace('SAD','').replace('ANGRY','')\
            .replace('AFRAID','').replace('SEE THE RESULTS','').replace(' \n','').replace('\n ','').replace('\n\n','').replace('  ','') 
        print(f'\n[Content]\n{article_content}')
        '''
        ### PREPARE SENTIMENT RESULTS
        # Scroll to 'SEE THE RESULTS' button then simulate click to get sentiment results
        sleep(3)
        sentiment_classname = self.driver.find_element(By.XPATH, '//*[@id="__next"]/div/div[16]/div[6]/div[4]/following::div').get_attribute('class')
        self.driver.execute_script(f'document.getElementsByClassName("{sentiment_classname}").scrollIntoView', '')
        
        # Automate click event to get sentiment results
        sentiment_index = 0
        for i in range(100):
            try:
                btn_sentiment = self.driver.find_element(By.XPATH, f'//*[@id="__next"]/div/div[16]/div[6]/div[4]/div[{i}]/div/div/span')
                if btn_sentiment.text.upper() == 'SEE THE RESULTS':
                    sleep(2)
                    self.close_popups()
                    action = ActionChains(self.driver)
                    action.move_to_element(btn_sentiment).click().perform()
                    # btn_sentiment.click()
                    sentiment_index = i
                    break
            except:
                pass

        sleep(3)
        # Clean some sentiment texts
        sentiment_results = self.driver.find_element(By.XPATH, f'//*[@id="__next"]/div/div[16]/div[6]/div[4]/div[{sentiment_index}]/div')
        sentiment_results = sentiment_results.text.replace('HOW OTHERS ARE FEELING', '').replace(' \n','').replace('\n ','').replace(' ARE ','-')
        # Convert each line of sentiment text to a list item
        sentiment_results = list(filter(None, sentiment_results.split('\n')))
        # Make a key-value pair of sentiment text and store it as dictionary object
        sentiment_results = {s.split('-')[1] : float(s.split('-')[0].split('%')[0])*.01 for s in sentiment_results}
        # Add a 0% value for sentiment key that does not have results in the current article and store as 'other sentiments'
        sentiment_keys = ['INSPIRED', 'HAPPY', 'AMUSED', 'NEUTRAL', 'ANNOYED', 'SAD', 'ANGRY', 'AFRAID']
        sentiment_others = {k : 0. for k in sentiment_keys if k not in sentiment_results}
        # Combine 'other sentiment dictionary' to the 'sentiment result dictionary' 
        sentiment_results = {**sentiment_results, **sentiment_others}
        print(f'\n[Sentiment]')
        print(sentiment_results)
        '''

        ### GET SENTIMENT RESULTS FROM ARTICLE
        return {'urls':url, 'titles':title, 'authors':author, 'published_dates':pubdate, 'articles_content':article_content}
    

# RUN WEB SCRAPING

In [8]:
SEARCH_URL = f'https://www.inquirer.net/search?q=covid%2019%20vaccine#gsc.tab=0&gsc.q=covid%2019%20vaccine%20philippines&gsc.sort=&gsc.page='

scraper = InquirerScraper(SEARCH_URL)
scraper.crawl()

dataset = []
print(f'\nTotal number of articles: {scraper.get_urls_count()}')
for i,url in enumerate(scraper.get_urls()):
    try:
        print(f'\n\n\n[{i+1}]{" ":>5}SCRAPING DATA  ~>  {url}')
        data = scraper.scrape_data(url)
        dataset.append(data)
    except Exception as e:
        print(f'SCRAPING ERROR: {e}')
        pass

scraper.get_driver().quit()

Total number of pages to crawl: 10

Start crawling article URLs for page 1...
Done.
Start crawling article URLs for page 2...
Done.
Start crawling article URLs for page 3...
Done.
Start crawling article URLs for page 4...
Done.
Start crawling article URLs for page 5...
Done.
Start crawling article URLs for page 6...
Done.
Start crawling article URLs for page 7...
Done.
Start crawling article URLs for page 8...
Done.
Start crawling article URLs for page 9...
Done.
Start crawling article URLs for page 10...
Done.
Start crawling article URLs for page 11...
Done.

Total number of articles: 104



[1]     SCRAPING DATA  ~>  https://newsinfo.inquirer.net/1404600/ph-so-far-administered-40000-vaccine-doses-vs-covid-19

[Title]
PH has so far vaccinated 44,000 against COVID-19 – Galvez

[Date Pulished]
INQUIRER.net / 12:25 AM March 09, 2021

[Author]
Daphne Galvez

[Content]
MANILA, Philippines — Over 44,000 vaccine doses against COVID-19 have so far been administered in the Philippines since it


[Title]
20 million doses of COVID-19 vaccines may arrive in 2nd quarter of 2021 – Galvez

[Date Pulished]
INQUIRER.net / 12:09 AM March 09, 2021

[Author]
Krissy Aguilar

[Content]
MANILA, Philippines — The Philippines may get more than 20 million doses of COVID-19 vaccines in the second quarter of 2021 — that is April, May, or June, Secretary Carlito Galvez Jr., vaccine czar and National Task Force against COVID-19 chief implementer, said Monday in a pre-recorded briefing with President Rodrigo Duterte.
Although he did not detail the number of doses for each brand, Galvez said these 20 million doses would come from Sinovac BioTech, the World Health Organization-led COVAX facility, Novovax, and AstraZeneca.The 2.6 million doses of AstraZeneca ordered by the private sector might arrive in May or June, Galvez said.
The Philippine government and the private sector signed an agreement to buy those doses with AstraZeneca in November 2020.
By the second quarter of this year, the government 


[Title]
COVID-19 vaccine bill now up for Duterte’s signature

[Date Pulished]
INQUIRER.net / 10:07 PM February 23, 2021

[Author]
Christia Marie Ramos

[Content]
MANILA, Philippines — A bill seeking to speed up the procurement and administration of COVID-19 vaccines in the country and establish an indemnity fund now only needs the signature of President Rodrigo Duterte.
This, after the House of Representatives, adopted the Senate’s version of the proposed Covid-19 Vaccination Program Act of 2021.During the House session on Tuesday, the lower chamber approved a motion to adopt Senate Bill No. 2057 as an amendment to House Bill No. 8648, passed on Monday.
READ: House OKs bill expediting Covid-19 vaccine purchase, creating indemnification fund
With the House’s concurrence with the Senate bill, a bicameral conference committee will no longer be needed to reconcile the disagreeing provisions of the chambers’ respective versions of the measure.
Under Senate Bill No. 2057, the national gover


[Title]
Duterte will get COVID-19 vaccine shot in private

[Date Pulished]
Philippine Daily Inquirer / 04:53 AM January 20, 2021

[Author]
Leila B. Salaverria

[Content]
MANILA, Philippines — President Rodrigo Duterte will be given the COVID-19 vaccine once it becomes available in the Philippines, but he will be vaccinated behind closed doors, Malacañang said on Tuesday.
Presidential spokesperson Harry Roque said the public would be informed if the vaccine had been administered to Mr. Duterte.Roque was answering a question about the President’s willingness to take the first shot of CoronaVac, the COVID-19 vaccine made by the Chinese company Sinovac Biotech.
‘No problem’
He said he had talked to Mr. Duterte about the question and the President had responded, “No problem, I will take the vaccine as soon as it is available.”
The President knows he has to be inoculated against COVID-19, Roque said.
“But he said it does not have to be shown to the public. He is taking the route of the Brit


[Title]
Pfizer, Sinovac deliveries to PH face more delays

[Date Pulished]
Philippine Daily Inquirer / 05:32 AM February 19, 2021

[Author]
Leila B. Salaverria

[Content]
MOCK VACCINATION About a hundred elderly people take part in a dry run of COVID-19 vaccination at Legarda Elementary School in Manila on Thursday. The practice is held in preparation for the rollout of Pfizer and Sinovac vaccines expected to arrive this month, but the deliveries will be delayed due to paperwork for indemnification required by the World Health Organization-led vaccine pool for Pfizer and by China’s requirement of emergency use authorization for Sinovac. —RICHARD A. REYES
The delivery of Pfizer and Sinovac COVID-19 vaccines to the country, scheduled for this month, is facing more delays.
Even if the Philippines has submitted the indemnification agreement required by the COVAX vaccine pool of the World Health Organization (WHO), Pfizer-BioNTech still has to submit its own indemnification agreement that 

...


[Title]
PH, ADB in talks for $400-500M vaccine procurement financing

[Date Pulished]
Inquirer Business / 03:45 PM December 16, 2020

[Author]
Ben O. de Vera

[Content]
MANILA, Philippines — The Asian Development Bank (ADB) will lend to its host-country the Philippines at least $9.4 billion — possibly including about $400-500 million for COVID-19 vaccine procurement — in the next three years so the economy can bounce back from what seemed to be a slow recovery from the pandemic-induced recession so far.
ADB vice president Ahmed M. Saeed said in a statement Wednesday, about two-thirds of the Manila-based multilateral lender’s Country Operations Business Plan 2021–2023 for the Philippines will support financing for infrastructure development, health care, and jobs recovery.Oscar Amiel A. Badiola, senior programs officer at the ADB’s Philippines country office, told a press briefing Wednesday that the lender extended to the Philippines a record-high $4.2 billion across 11 loans, of which


[Title]
Duterte wants sure supply of COVID-19 vaccines from US, not freebies — Palace

[Date Pulished]
INQUIRER.net / 11:13 AM December 27, 2020

[Author]
Krissy Aguilar

[Content]
President Rodrigo Duterte
MANILA, Philippines — President Rodrigo Duterte was not asking for free vaccines, but only wanted the Philippines to have a guaranteed share when he threatened to continue with the abrogation of the Visiting Forces Agreement (VFA) if the US could not provide its COVID-19 vaccines to the country, Malacañang clarified on Sunday.
Duterte, in a speech on Saturday night, demanded at least 20 million doses of coronavirus vaccines from the U.S. to lift the termination of the VFA. 
“Hindi po, babayaran po natin. Ang problema lang po, yung supply. Ang sinasabi ng Presidente, at least bigyan niyo kami ng supply. Hindi naman tayo nanlilimos, meron naman tayong perang pambayad dyan at babayaran natin,” presidential spokesperson Harry Roque said in an interview over Teleradyo.
(No, we will pay.


[Title]
AstraZeneca eyes clinical trials for its COVID-19 vaccine in PH

[Date Pulished]
INQUIRER.net / 01:52 PM November 25, 2020

[Author]
Darryl John Esguerra

[Content]
FILE PHOTO: Small bottles labelled with “Vaccine” stickers stand near a medical syringe in front of displayed “Coronavirus COVID-19” words in this illustration taken April 10, 2020. REUTERS/Dado Ruvic/Illustration
MANILA, Philippines — British drug maker AstraZeneca has already submitted the necessary documents to Philippine health authorities to secure approval in conducting clinical trials for its COVID-19 vaccine in the country.
Health Undersecretary Maria Rosario Vergeire said this Wednesday, just days after AstraZeneca announced that its COVID-19 vaccine can be 90% effective.“They already submitted their documents to our Vaccine Experts Panel last November 16, during that time may kulang pa [they still have deficiencies],” Vergeire told reporters in an online media briefing.
“Immediately, last November 18, the


[Title]
Poor infra to delay COVID-19 vaccination in rural PH

[Date Pulished]
Philippine Daily Inquirer / 04:07 AM February 24, 2021
SCRAPING ERROR: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="art_author"]/span/a"}
  (Session info: chrome=89.0.4389.82)




[101]     SCRAPING DATA  ~>  https://globalnation.inquirer.net/194085/ph-reserves-6-m-doses-of-johnson-johnsons-covid-19-vaccine

[Title]
PH reserves 6 M doses of Johnson & Johnson’s Covid-19 vaccine

[Date Pulished]
INQUIRER.net / 08:27 AM March 01, 2021

[Author]
Christia Marie Ramos

[Content]
A vial and sryinge are seen in front of a displayed Johnson&Johnson logo in this illustration taken January 11, 2021. REUTERS/Dado Ruvic/Illustration/File Photo
MANILA, Philippines — The Philippines has reserved six million doses of the Covid-19 vaccine developed by Johnson & Johnson, Manila’s top diplomat in the United States said Monday.
Philippine Ambassador to the U.S. Jose Romualdez, the P


[Title]
Top US official will try to get back fraction of Pfizer’s COVID vaccine for PH — Locsin

[Date Pulished]
INQUIRER.net / 11:48 AM December 19, 2020

[Author]
Katrina Hallare

[Content]
FILE PHOTO: A woman holds a small bottle labeled with a “Coronavirus COVID-19 Vaccine” sticker and a medical syringe in front of displayed Pfizer logo in this illustration taken, October 30, 2020. REUTERS/Dado Ruvic/File Photo
MANILA, Philippines — U.S. Secretary of State Mike Pompeo will “give his best try”to get even a fraction of the 10 million doses of Pfizer’s COVID-19 vaccine for the Philippines, Foreign Affairs Secretary Teodoro. Locsin Jr. said aftera recent conversation with the ranking US government official.
Great phone conversation with Mike Pompeo last night. Lotsa laughs at others. But on the most serious note I asked him to help Babe and I get back even a fraction of the 10 million doses of Pfizer after someone dropped the ball. He’ll give it his best try. Babe’s securing Moderna
—

# EDA

In [9]:
df = pd.DataFrame(dataset)
df

,urls,titles,authors,published_dates,articles_content
0,https://newsinfo.inquirer.net/1404600/ph-so-fa...,"PH has so far vaccinated 44,000 against COVID-...",Daphne Galvez,"INQUIRER.net / 12:25 AM March 09, 2021","MANILA, Philippines — Over 44,000 vaccine dose..."
1,https://newsinfo.inquirer.net/1372054/explaine...,Explainer: Facts about 7 COVID-19 vaccines Phi...,Cristina Eloisa Baclig,"INQUIRER.net / 08:45 PM December 15, 2020","MANILA, Philippines — As the country pursues a..."
2,https://newsinfo.inquirer.net/1404592/20-milli...,20 million doses of COVID-19 vaccines may arri...,Krissy Aguilar,"INQUIRER.net / 12:09 AM March 09, 2021","MANILA, Philippines — The Philippines may get ..."
3,https://opinion.inquirer.net/136192/covid-19-v...,COVID-19 vaccines in the Philippine context,Gideon Lasco,Philippine Daily Inquirer / 05:05 AM December ...,"Several months and over 400,000 COVID-19 cases..."
4,https://newsinfo.inquirer.net/1399299/covid-19...,COVID-19 vaccine bill now up for Duterte’s sig...,Christia Marie Ramos,"INQUIRER.net / 10:07 PM February 23, 2021","MANILA, Philippines — A bill seeking to speed ..."
...,...,...,...,...,...
90,https://newsinfo.inquirer.net/1401139/coronava...,CoronaVac rollout begins; AstraZeneca delayed,Maricar Cinco,"Philippine Daily Inquirer / 05:00 AM March 01,...",FINALLY The Philippines’ vaccination drive sta...
91,https://globalnation.inquirer.net/194085/ph-re...,PH reserves 6 M doses of Johnson & Johnson’s C...,Christia Marie Ramos,"INQUIRER.net / 08:27 AM March 01, 2021",A vial and sryinge are seen in front of a disp...
92,https://globalnation.inquirer.net/190193/duter...,Duterte says Russia may give COVID-19 vaccine ...,Krissy Aguilar,"INQUIRER.net / 03:52 PM August 11, 2020","MANILA, Philippines — Russia is willing to pro..."
93,https://globalnation.inquirer.net/192800/top-u...,Top US official will try to get back fraction ...,Katrina Hallare,"INQUIRER.net / 11:48 AM December 19, 2020",FILE PHOTO: A woman holds a small bottle label...


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 95 entries, 0 to 94
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   urls              95 non-null     object
 1   titles            95 non-null     object
 2   authors           95 non-null     object
 3   published_dates   95 non-null     object
 4   articles_content  95 non-null     object
dtypes: object(5)
memory usage: 3.8+ KB


In [11]:
df.describe().T

,count,unique,top,freq
urls,95,95,https://globalnation.inquirer.net/194031/fwd-e...,1
titles,95,95,Cavite allots P750 million for COVID-19 vaccines,1
authors,95,19,Krissy Aguilar,22
published_dates,95,95,Philippine Daily Inquirer / 04:53 AM January 2...,1
articles_content,95,95,"MANILA, Philippines — There’s a “compelling le...",1


In [12]:
df.isna().sum()

urls                0
titles              0
authors             0
published_dates     0
articles_content    0
dtype: int64

In [13]:
### Save a CSV dataset
df.to_csv('inquirer_vaccine.csv', index=False)

# WORDCLOUD

In [72]:
from wordcloud import WordCloud
from PIL import Image
import numpy as np

ImportError: DLL load failed while importing query_integral_image: The specified module could not be found.

In [68]:
import sys
print(sys.executable)


C:\Users\gear\AppData\Local\Programs\Python\Python39\python.exe


In [ ]:
def grey_color_func(word, font_size, position, orientation, random_state=None, **kwargs):
    return "hsl(0, 0%%, %d%%)" % np.random.randint(60, 100)

mask = np.array(Image.open('bg.png'))

words = ' '.join(df['articles_content'].tolist())
wordcloud = WordCloud(background_color='black', max_words=2000, width=1024, height=720, mask=mask)
wordcloud.generate(words)

plt.figure(figsize=(20, 8), dpi=300)
plt.title('Rappler on COVID Vaccine in PH (Word Cloud)')
plt.imshow(wordcloud.recolor(color_func=grey_color_func, random_state=42), interpolation='bilinear')
plt.axis('off')
plt.show()